#### *ISEL - DEI - LEIM*
## Aprendizagem Automática [T52D]
### Trabalho Laboratorial 2: Classificação de Críticas de Cinema do IMDb

João Madeira ($48630$), 
Renata Góis ($51038$),
Bruno Pereira ($51811$)

**Docentes responsáveis:** 
- Prof. Gonçalo Xufre Silva

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as p
import re
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
with open("resources/imdbFull.p", "rb") as f:
    D = p.load(f)
print("Keys:", D.keys())

reviews = D['data']
sentiments = D['target']

print(len(reviews), "reviews")

Keys: dict_keys(['data', 'target', 'DESCR'])
50000 reviews


This dataset contains movie reviews along with their associated binary sentiment polarity labels. It is intended to serve as a benchmark for sentiment classification. This document outlines how the dataset was gathered, and how to use the files provided.
For more details see: http://ai.stanford.edu/~amaas/data/sentiment/

GPT answear when I asked what are my options for stemmers

| Method                   | Aggressiveness | Quality | Speed  | Best for                    |
| ------------------------ | -------------- | ------- | ------ | --------------------------- |
| **Porter**               | Medium         | ✔✔      | Fast   | Classic NLP                 |
| **Snowball**             | Medium         | ✔✔✔     | Fast   | Best stemmer for English    |
| **Lancaster**            | High           | ✔       | Fast   | Rare cases; very aggressive |
| **Lemmatizer (spaCy)**   | Low            | ⭐⭐⭐⭐    | Medium | Best accuracy               |
| **Lemmatizer (WordNet)** | Low            | ⭐⭐⭐     | Medium | Simpler lemmatization       |

So I opted for the "Best stemmer for english" since that what we are doing and for what I gathered Lemmatizer $ \not= $ Stemmer

In [ ]:
stemmer = SnowballStemmer("english")

def clean_review(string):
    # remove tags html
    string = string.replace('<br />', ' ')  

    # remove palavras demasiado longas
    string = re.sub(r'\b[a-zA-Z]{20,}\b', ' ', string)

    # remove coisas tipo yaaass, omgggg, etc.
    string = re.sub(r'\b\w*(.)\1{2,}\w*\b', ' ', string)

    # filtra apenas letras
    string = re.sub(r'[^a-zA-Z]', ' ', string)

    # remove espaços extra
    string = re.sub(r'\s+', ' ', string).strip()

    # lowercase só no fim
    string = string.lower()

    # stemming
    string = " ".join(stemmer.stem(w) for w in string.split())
    return string

reviews = [clean_review(rev) for rev in reviews]

output = {"data" : reviews, "target" : sentiments}
p.dump(output,open("resources/pre_processed_data.p",'wb'))

In [27]:
tfidVector = TfidfVectorizer(min_df=10,                   #Remove palavras que aparecem menos de 10 vezes no dataset
                        max_df=0.9,                       #Remove palavras que aparecem em 90% do dataset
                        ngram_range=(1,2),                #Utiliza unigramas e bigramas (good, very good, pretty bad)
                        token_pattern=r'\b[a-zA-Z]{3,}\b',#Ignora palavras com menos de 3 letras
                        sublinear_tf=True                 #term frequancy passa a ter um comportamento logarítmico em vez de linear
                        )

In [33]:
pre_processed_data = p.load(open("resources/pre_processed_data.p","rb"))
reviews = pre_processed_data['data']
sentiments = pre_processed_data['target']
tfidVector = tfidVector.fit(reviews)
tokens = tfidVector.get_feature_names_out()
X = tfidVector.transform(reviews)
len(tokens)

117374